# Plotly Examples

For this project, we chose to use plotly to create our visualizations. The reasoning behind this is detailed in the technology presentation presentation located in the docs folder of the repo. 

In [26]:
import pandas as pd
import plotly.express as px 
import plotly.io as pio
import plotly.graph_objects as go
denied_boarding = pd.read_csv("..\\SmoothFly\\analysis\\clean_data\\denied_boarding.csv")
denied_boarding_reshape = pd.melt(denied_boarding.rename(columns = {
        '1':'Involuntary - Received Compensation', 
        '2':'Involuntary - No Compensation', 
        '5':'Voluntary - Received Compensation'
        }),
                                      id_vars = ['CARRIER', 'year', 'quarter'],
                                      value_vars=['Involuntary - Received Compensation',
                                                  'Involuntary - No Compensation',
                                                  'Voluntary - Received Compensation'
                                                  ])

denied_boarding_reshape = denied_boarding_reshape[['CARRIER',
                                  'variable',
                                  'value']].set_index(['CARRIER', 'variable'])['value']

pio.templates.default = "plotly"


For these examples, we will use the denied_boarding dataset that is located in the clean_data folder of the repo. Plotly express can be used to create a variety of graphs with customization and interactive elements.If we wanted to use plotly to visualize the percent of people denied boarding over time we could make a standard area plot using the code below. 

In [13]:
fig = px.area(denied_boarding.groupby('date').mean(numeric_only = True).
                  reset_index().rename(columns = {'perc_denied_boarding_involuntary':"Involuntary",
                                                  "perc_denied_boarding_voluntary":"Voluntary"}),
              x="date", y=["Voluntary","Involuntary"])
fig.show()

In order to make this plot more interpretable we can use plotly features such as labels and title. The code below creates the same plot, but with additional descriptors to make it clear what the visualization is showing. 

In [14]:
fig = px.area(denied_boarding.groupby('date').mean(numeric_only = True).
                  reset_index().rename(columns = {'perc_denied_boarding_involuntary':"Involuntary",
                                                  "perc_denied_boarding_voluntary":"Voluntary"}),
              x="date", y=["Voluntary","Involuntary"],
                labels={
                    "date": "Year",
                     "value":"Percent of Passengers Denied Boarding", 
                     "variable":"Denial Type"
                 },
                title="Percent of Passengers Denied Boarding over Time")
fig.show()

Plotly also allows for customization of colors and formatting. In the following example we will adjust the tick marks so that they are angled for ease of reading and change the color palette. 

In [27]:
fig = px.area(denied_boarding.groupby('date').mean(numeric_only = True).
                  reset_index().rename(columns = {'perc_denied_boarding_involuntary':"Involuntary",
                                                  "perc_denied_boarding_voluntary":"Voluntary"}),
              x="date", y=["Voluntary","Involuntary"],
                labels={
                    "date": "Year",
                     "value":"Percent of Passengers Denied Boarding", 
                     "variable":"Denial Type"
                 },
                title="Percent of Passengers Denied Boarding over Time",
                color_discrete_sequence=px.colors.qualitative.Dark24)
fig.update_xaxes(tickangle = 45)
fig.show()

For one final customization component, plotly io allows users to set a template for their plots. The template can be changed in the plot or globally. Once a template has been assigned globally, the template will be used for all plotly figures, unless a template is explicitly assigned in the figure. First we will assign a global template and create a figure, and then we will assign a template within a plotly figure that will override the global value. 

In [25]:
pio.templates.default = "plotly_dark"

fig = px.area(denied_boarding.groupby('date').mean(numeric_only = True).
                  reset_index().rename(columns = {'perc_denied_boarding_involuntary':"Involuntary",
                                                  "perc_denied_boarding_voluntary":"Voluntary"}),
              x="date", y=["Voluntary","Involuntary"],
                labels={
                    "date": "Year",
                     "value":"Percent of Passengers Denied Boarding", 
                     "variable":"Denial Type"
                 },
                title="Percent of Passengers Denied Boarding over Time",
                color_discrete_sequence=px.colors.qualitative.Dark24)
fig.update_xaxes(tickangle = 45)
fig.show()


In [24]:
fig = px.area(denied_boarding.groupby('date').mean(numeric_only = True).
                  reset_index().rename(columns = {'perc_denied_boarding_involuntary':"Involuntary",
                                                  "perc_denied_boarding_voluntary":"Voluntary"}),
              x="date", y=["Voluntary","Involuntary"],
                labels={
                    "date": "Year",
                     "value":"Percent of Passengers Denied Boarding", 
                     "variable":"Denial Type"
                 },
                title="Percent of Passengers Denied Boarding over Time",
                color_discrete_sequence=px.colors.qualitative.Dark24, 
                template = 'seaborn')
fig.update_xaxes(tickangle = 45)
fig.show()

Plotly also allows users to create interactive visualizations. In this project we utilized this functionality to create drop down menus in our visualizations. The example below shows how you can first create a figure with different levels for each variable you want to allow a user to select from. In this case, we used airline carrier. When outputting this graph, we will see only the default figure which is the first carrier, because there is no drop down menu yet.  

In [29]:
figs = {c: px.pie(denied_boarding_reshape.loc[c].reset_index(), values="value", names="variable",
                  labels = {
                     'value':'Total Passengers', 
                     'variable':'Denial Type'
                 },
            color_discrete_sequence=px.colors.qualitative.Dark24).update_traces(
            name=c, visible=False,
        )
        for c in denied_boarding_reshape.index.get_level_values("CARRIER").unique()
    }

defaultcat = denied_boarding_reshape.index.get_level_values("CARRIER").unique()[0]
fig = figs[defaultcat].update_traces(visible=True)
for k in figs.keys():
    if k != defaultcat:
        fig.add_traces(figs[k].data)

fig.show()

In order to add a drop down menu, we will use the update_layout() function in plotly express to create a button with all the different values for carrier. The resulting figure will have a dropdown menu where a user can select the carrier they are interested in and plotly will return the figure associated with that airline. 


In [30]:
fig.update_layout(
    updatemenus=[
        {
            "buttons": [
                {
                    "label": k,
                    "method": "update",
                    # list comprehension for which traces are visible
                    "args": [{"visible": [kk == k for kk in figs.keys()]},
                             {"title":go.layout.xaxis.Title(
            text=f"Denied Boardings by Airline Carrier <br><sup>{k} </sup>"
            )}],
                }
                for k in figs.keys()
            ]
        }
    ]
)